In [ ]:
def lgb_train(train_, test_, feature_list):
    print(f'Start train lgb model, file describe: ', train_.shape, test_.shape)
    params = {
        'learning_rate': 0.01,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 62,
        'feature_fraction': 0.7,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'seed': 1,
        'bagging_seed': 3,
        'feature_fraction_seed': 2,
        'min_data_in_leaf': 20,
        'verbose': -1,
        'nthread': -1
    }
    n_split = 5
    X, y = train_[feature_list], train_['label']
    skf = StratifiedKFold(n_splits=n_split, random_state=1024, shuffle=True)
    roc_list = []
    sub_prob = np.zeros((test_.shape[0], n_split))
    feat_imp = pd.DataFrame()
    feat_imp['features'] = feature_list
    for index, (train_index, test_index) in enumerate(skf.split(X, y), start=1):
        print(f'the {index} training start ...')
        X_train, X_valid, y_train, y_valid = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        print(X_train.shape, X_valid.shape)
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid)
        gbm = lgb.train(params, train_data,
                        num_boost_round=5000,
                        valid_sets=valid_data,
                        early_stopping_rounds=100,
                        verbose_eval=100)
        feat_imp[f'Fold_{index}_imp'] = gbm.feature_importance()
        x_pred = gbm.predict(X_valid, num_iteration=gbm.best_iteration)
        roc_list.append(roc_auc_score(y_valid, x_pred))
        sub_prob[:, index - 1] = gbm.predict(test_[feature_list], num_iteration=gbm.best_iteration)
    feat_imp['imp_mean'] = (feat_imp[f'Fold_1_imp'] + feat_imp[f'Fold_2_imp'] + feat_imp[f'Fold_3_imp'] +
    feat_imp[f'Fold_4_imp'] + feat_imp[f'Fold_5_imp']) / 5
    feat_imp.to_csv('LBG1.csv', index=False, encoding='gbk')
    test_['prob'] = np.mean(sub_prob, axis=1)
    date = time.strftime('%Y%m%d_%H%M%S')
    test_[['id', 'prob']].to_csv(f'LGB{date}.txt', encoding='utf8', index=False)
    print('submit file successful generate ...')
    print(f'{n_split} folds: {roc_list}, avg: {np.mean(roc_list)}')